얼리엑세스검색후

포스트 제목 + 코멘트 가져오기


In [29]:
import csv
from requests import get
from bs4 import BeautifulSoup

In [31]:
def search_in_reddit(total_urls):
    headers = {'User-Agent': 'Mozilla/5.0'}
    results = []
    for url in total_urls:
        page = get(url, headers=headers)
        if page.status_code != 200:
            print(f"Error: Could not retrieve {url}")
            continue
        soup = BeautifulSoup(page.text, 'html.parser')
        titles = soup.find_all("header", class_="search-result-header")
        for title_section in titles:
            title_post = title_section.find('a')
            post_url = title_post['href']
            post_page = get(post_url, headers=headers)
            soup_page = BeautifulSoup(post_page.text, 'html.parser')
            page_titles = soup_page.find("a", class_="title may-blank ")
            if page_titles:
                page_titles_text = page_titles.text
            else:
                page_titles_text = ""
            page_content = soup_page.find_all('div', class_="md")
            page_content.pop(0)
            content_result = []
            comments_result = []
            is_content = True
            for content in page_content:
                content_section = content.find_all('p')
                if is_content:
                    content_result.append(content_section)
                    is_content = False
                else:
                    comments_result.append(content_section)
            results.append({'Title': page_titles_text, 'Content': content_result, 'Comments': comments_result})
    return results



In [30]:
total_urls = ['https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all', 
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=25&after=t3_val90l',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=50&after=t3_az23ml',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=75&after=t3_fr3d5z',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=100&after=t3_8u9dx6',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=125&after=t3_7hioz9',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=150&after=t3_1i3k22',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=175&after=t3_9poafo',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=200&after=t3_3nho6g',
              'https://old.reddit.com/r/patientgamers/search?q=early+access&restrict_sr=on&sort=relevance&t=all&count=225&after=t3_oeafx1']

results = search_in_reddit(total_urls)

# Save results to a CSV file
with open('reddit_posts_final.csv', 'w', newline='', encoding='utf-8') as file:
    fieldnames = ['Title', 'Content', 'Comments']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for row in results:
        writer.writerow(row)